# 1. Data Ingestion

In [1]:
import os

In [2]:
!pwd

/c/Users/lavanyaa/Projects/End-to-End-Project-ML-Flow/research


In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\lavanyaa\\Projects\\End-to-End-Project-ML-Flow'

In [19]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir: Path


# #Constant - paths the yaml file

# from pathlib import Path

# CONFIG_FILE_PATH=Path("config/config.yaml")
# PARAMS_FILE_PATH=Path("params.yaml")
# SCHEMA_FILE_PATH = Path("schema.yaml")


In [20]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [ ]:
# Component

import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL, filename=self.config.local_data_file)
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        # print("ZIP File Path:", self.config.local_data_file)
        # print("File Size (bytes):", os.path.getsize(self.config.local_data_file))
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
#Pipeline

try:
    config=ConfigurationManager()
    data_ing_config=config.get_data_ingestion_config()
    data_ing=DataIngestion(config=data_ing_config)
    data_ing.download_file()
    data_ing.extract_zip_file()
except Exception as e:
    raise e

[2025-06-17 08:39:22,351:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-06-17 08:39:22,351:INFO:common: yaml file: params.yaml loaded successfully]
[2025-06-17 08:39:22,360:INFO:common: yaml file: schema.yaml loaded successfully]
[2025-06-17 08:39:22,360:INFO:common: created directory at: artifacts]
[2025-06-17 08:39:22,360:INFO:common: created directory at: artifacts/data_ingestion]
[2025-06-17 08:39:23,953:INFO:1184442725: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B3EB:170B30:6BAC9:187273:6850DC9D
Accept-Ranges: bytes
Date: Tue, 17 Jun 2025 03:1

# Data validation

In [ ]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [30]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(root_dir=config.root_dir,
                                                     STATUS_FILE=config.STATUS_FILE,
                                                     unzip_data_dir=config.unzip_data_dir,
                                                     all_schema=schema)
        return data_validation_config

In [31]:
import os
from mlProject import logger
import pandas as pd

In [32]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config=config

    def validate_all_columns(self)-> bool:
        try:
            validation_status=True
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation Status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e

In [33]:
try:
    config = ConfigurationManager()
    data_val_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_val_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-18 15:09:25,038:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-06-18 15:09:25,040:INFO:common: yaml file: params.yaml loaded successfully]
[2025-06-18 15:09:25,043:INFO:common: yaml file: schema.yaml loaded successfully]
[2025-06-18 15:09:25,045:INFO:common: created directory at: artifacts]
[2025-06-18 15:09:25,047:INFO:common: created directory at: artifacts/data_validation]


# Data Transformation

In [34]:
@dataclass(frozen= True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [35]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(root_dir=config.root_dir,
                                                     STATUS_FILE=config.STATUS_FILE,
                                                     unzip_data_dir=config.unzip_data_dir,
                                                     all_schema=schema)
        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(root_dir=config.root_dir,
                                                              data_path=config.data_path)
        return data_transformation_config

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
# Components


class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config=config
    
    def train_test_spliting(self):
        data=pd.read_csv(self.config.data_path)
        train,test=train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info('Splitted data into training and test sets')
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


In [40]:
try:
    config=ConfigurationManager()
    data_trans_config=config.get_data_transformation_config()
    data_trans=DataTransformation(config=data_trans_config)
except Exception as e:
    raise e

[2025-06-18 17:52:09,765:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-06-18 17:52:09,771:INFO:common: yaml file: params.yaml loaded successfully]
[2025-06-18 17:52:09,782:INFO:common: yaml file: schema.yaml loaded successfully]
[2025-06-18 17:52:09,800:INFO:common: created directory at: artifacts]
[2025-06-18 17:52:09,806:INFO:common: created directory at: artifacts/data_transformation]


# 4. Model Training

In [47]:
# Entity
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column:str

In [48]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(root_dir=config.root_dir,
                                                     STATUS_FILE=config.STATUS_FILE,
                                                     unzip_data_dir=config.unzip_data_dir,
                                                     all_schema=schema)
        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(root_dir=config.root_dir,
                                                              data_path=config.data_path)
        return data_transformation_config
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(root_dir=config.root_dir,
                                                 train_data_path=config.train_data_path,
                                                 test_data_path=config.test_data_path,
                                                 model_name=config.model_name,
                                                 alpha=params.alpha,
                                                 l1_ratio=params.l1_ratio,
                                                 target_column=schema.name)
        return model_trainer_config

In [44]:
from sklearn.linear_model import ElasticNet
import joblib

In [49]:
# Component

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio,
                      random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [51]:
#pipeline
try:
    config = ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-06-18 18:14:33,227:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-06-18 18:14:33,230:INFO:common: yaml file: params.yaml loaded successfully]
[2025-06-18 18:14:33,234:INFO:common: yaml file: schema.yaml loaded successfully]
[2025-06-18 18:14:33,236:INFO:common: created directory at: artifacts]
[2025-06-18 18:14:33,237:INFO:common: created directory at: artifacts/model_trainer]


# 5. Model Evaluation

In [ ]:
from dotenv import load_dotenv
load_dotenv()

username = os.getenv("DAGSHUB_USERNAME")
token = os.getenv("DAGSHUB_TOKEN")


In [54]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [ ]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(root_dir=config.root_dir,
                                                     STATUS_FILE=config.STATUS_FILE,
                                                     unzip_data_dir=config.unzip_data_dir,
                                                     all_schema=schema)
        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(root_dir=config.root_dir,
                                                              data_path=config.data_path)
        return data_transformation_config
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(root_dir=config.root_dir,
                                                 train_data_path=config.train_data_path,
                                                 test_data_path=config.test_data_path,
                                                 model_name=config.model_name,
                                                 alpha=params.alpha,
                                                 l1_ratio=params.l1_ratio,
                                                 target_column=schema.name)
        return model_trainer_config
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(root_dir=config.root_dir,
                        test_data_path = config.test_data_path,
                        model_path=config.model_path,
                        all_params=params,
                        metric_file_name=config.metric_file_name,
                        target_column=schema.name,
                        mlflow_uri= os.getenv("MLFLOW_TRACKING_URI"))
        return model_evaluation_config

In [62]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlProject.utils.common import save_json
import numpy as np

In [ ]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config=config
    
    def eval_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)
            (rmse,mae,r2) = self.eval_metrics(test_y, predicted_qualities)

            scores={'rmse':rmse, 'mae':mae,'r2':r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric('rmse',rmse)
            mlflow.log_metric('r2',r2)
            mlflow.log_metric('mae',mae)
            
            if tracking_url_type_store!='file':
                mlflow.sklearn.log_model(model,"model",registered_model_name='ElasticnetModel')
            else:
                mlflow.sklearn.log_model(model,"model")



In [64]:
#pipeline
try:
    config=ConfigurationManager()
    model_eval_config=config.get_model_evaluation_config()
    model_eval_config=ModelEvaluation(config=model_eval_config)
    model_eval_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-06-20 11:21:15,150:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-06-20 11:21:15,153:INFO:common: yaml file: params.yaml loaded successfully]
[2025-06-20 11:21:15,157:INFO:common: yaml file: schema.yaml loaded successfully]
[2025-06-20 11:21:15,159:INFO:common: created directory at: artifacts]
[2025-06-20 11:21:15,161:INFO:common: created directory at: artifacts/model_evaluation]
[2025-06-20 11:21:15,825:INFO:common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\lavanyaa\Miniconda3\envs\mlproj\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'ElasticnetModel'.
2025/06/20 11:21:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
